In [1]:
# !pip install folktables

In [2]:
#info about Data
# https://arxiv.org/pdf/2108.04884.pdf

#ACSDataSource ,ACSEmployment, ACSPublicCoverage, ACSIncome

-> (using ACSEmployment) from the paper: predict whether an individual is employed, after filtering the ACS PUMS data
sample to only include individuals between the ages of 16 and 90  

In [3]:
# !pip install torchsummary

In [4]:
# !python --version

In [1]:
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import pandas as pd
import time
import numpy as np
import copy
import glob
import sys, os
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import pickle
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import os
from torch.utils.data import Dataset
import numpy as np
import torch.nn.functional as F
import pandas as pd
from scipy.stats import multivariate_normal
import torch, random, copy, os

from torch.utils.data import DataLoader, Subset, random_split

In [6]:
# detail about the state: load_acs.py

In [7]:
from folktables import ACSDataSource, ACSEmployment,ACSIncome
data_source = ACSDataSource(survey_year='2017', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["AL"], download=True)

In [8]:
# features_np, label_np, group_np = ACSEmployment.df_to_numpy(acs_data)
features, label, group = ACSEmployment.df_to_pandas(acs_data)

In [9]:
features

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P
0,73.0,10.0,4.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0
1,31.0,21.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
3,48.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47640,20.0,18.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
47641,18.0,16.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
47642,25.0,17.0,1.0,16.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,9.0
47643,24.0,17.0,5.0,16.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0


In [10]:
label

,ESR
0,False
1,True
2,True
3,False
4,False
...,...
47640,False
47641,True
47642,False
47643,False


In [7]:
merged_df=pd.concat([features, label], axis=1)
merged_df.head()

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,73.0,10.0,4.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,False
1,31.0,21.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,True
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,True
3,48.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False


In [8]:
len(merged_df)

47645

In [9]:
group.value_counts()

RAC1P
1        34827
2        10641
9          821
6          617
8          468
3          207
5           47
7           15
4            2
Name: count, dtype: int64

In [10]:
merged_df["RAC1P"].value_counts()

# 1: White alone – 2: Black or African American alone

RAC1P
1.0    34827
2.0    10641
9.0      821
6.0      617
8.0      468
3.0      207
5.0       47
7.0       15
4.0        2
Name: count, dtype: int64

In [11]:
pd.unique(merged_df["RAC1P"])

array([2., 1., 9., 6., 3., 8., 5., 7., 4.])

In [12]:
#  for Black and white people
filtered_df= merged_df[["SEX","RAC1P","ESR"]][merged_df['RAC1P'].isin([1,2])]
print("len(filtered_df): ", len(filtered_df),"\n")
print(filtered_df["ESR"].value_counts(),"\n")
print(filtered_df["SEX"].value_counts(),"\n")
print(filtered_df["RAC1P"].value_counts(),"\n")
filtered_df.head() 

len(filtered_df):  45468 

ESR
False    27041
True     18427
Name: count, dtype: int64 

SEX
2.0    23697
1.0    21771
Name: count, dtype: int64 

RAC1P
1.0    34827
2.0    10641
Name: count, dtype: int64 



,SEX,RAC1P,ESR
0,1.0,2.0,False
1,2.0,1.0,True
2,1.0,1.0,True
3,2.0,1.0,False
4,1.0,1.0,False


In [13]:
# 1- Male 2- Female
# 1- White 2-Black

counts_frq = filtered_df.groupby(['SEX', 'RAC1P']).size()
counts_frq

SEX  RAC1P
1.0  1.0      16783
     2.0       4988
2.0  1.0      18044
     2.0       5653
dtype: int64

In [14]:
merged_df

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,73.0,10.0,4.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,False
1,31.0,21.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,True
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,True
3,48.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47640,20.0,18.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47641,18.0,16.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,True
47642,25.0,17.0,1.0,16.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,9.0,False
47643,24.0,17.0,5.0,16.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False


In [15]:
# # filter the dataframe for each combination

#White Men
WM = merged_df.loc[(merged_df['SEX'] == 1) & (merged_df['RAC1P'] == 1)]
#Black Men
BM = merged_df.loc[(merged_df['SEX'] == 1) & (merged_df['RAC1P'] == 2)]
#white Women
WW = merged_df.loc[(merged_df['SEX'] == 2) & (merged_df['RAC1P'] == 1)]
#Black Women
BW = merged_df.loc[(merged_df['SEX'] == 2) & (merged_df['RAC1P'] == 2)]

In [16]:
WM

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,True
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
10,32.0,16.0,1.0,0.0,2.0,0.0,5.0,3.0,4.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,True
18,80.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,False
19,32.0,22.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,4.0,1.0,2.0,2.0,2.0,1.0,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47624,18.0,19.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47632,29.0,14.0,5.0,16.0,2.0,0.0,1.0,3.0,4.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47638,18.0,19.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47640,20.0,18.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False


In [17]:
len(WM)+len(BM)+len(WW)+len(BW)

45468

In [19]:
list_client_df=[]

output_directory="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/client_df"
encoder = LabelEncoder()
BATCH=64


all_df_split=[WM,BM,WW,BW]
for idx, ind_df in enumerate(all_df_split):
    
    x_train=0
    y_train=0

    x_train = ind_df[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'SEX', 'RAC1P']]
    X_train_sens= ind_df[['SEX']]
    y_train = ind_df[['ESR']]
    
    # output_filename = os.path.join(output_directory, f"client_{idx}.csv")
    # ind_df.to_csv(output_filename, index=False)
    
    x_train = preprocessing.normalize(x_train)
    X_train_sens=X_train_sens['SEX'].map({1: 1, 2: 0}).to_numpy()
    
    encoder.fit(y_train)
    y_train = encoder.transform(y_train)
    
    X_train= torch.tensor(x_train, dtype=torch.float32)
    X_train_sens = torch.tensor(X_train_sens, dtype=torch.float32).reshape(-1, 1)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    
    final_data = data_utils.TensorDataset(X_train, y_train,X_train_sens)
    final_data1=data_utils.DataLoader(final_data, batch_size=BATCH, shuffle=True)
    
    list_client_df.append(final_data1)

/home/chiragapandav/Downloads/Hiwi/gpu_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chiragapandav/Downloads/Hiwi/gpu_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/chiragapandav/Downloads/Hiwi/gpu_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chiragapandav/Downloads/Hiwi/gpu_env/lib/python3.10/site-packages/sklearn/p

In [20]:
len(list_client_df)

4

In [21]:
import pickle

path = "/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/data_subG/"
client_names = ["WM", "BM", "WW", "BW"]

for i, client_df in enumerate(list_client_df):
    # Ensure we don't go out of bounds in the client_names list
    client_name = client_names[i % len(client_names)]
    
    client_filename = f"{client_name}_client_{i}_data.pkl"
    
    with open(path + client_filename, "wb") as f:
        pickle.dump(client_df, f)
